In [1]:
import pandas as pd 
import numpy as np
import os

# RUPTL Data

### Load the dataset

In [2]:
directory = r'C:\Users\binta\OneDrive\Documents\IESR\Generator\Datasets'
filepaths = [f for f in os.listdir(directory) if f.endswith('.csv')]

# Load the total capacity data for each province
capacity = pd.read_csv('TotalCapacity.txt', sep=';')

# Extract the province names 
provinces = capacity['Province'].values

# Check if the file names match the province names
print(f"{np.sum(np.array([f[3:-4] for f in filepaths] == provinces))}/34 match")

34/34 match


In [3]:
# Node data
nodes = pd.read_csv(r'C:\Users\binta\OneDrive\Documents\IESR\Generator\Node.csv', sep=';')

# Split elements by semicolon and flatten the list, ignoring nan values
node_province = np.array(sorted([prov for item in nodes['Coverage'].values if isinstance(item, str) for prov in item.split(";")], reverse=False))

# Map each province to the corresponding node
# Prepare the output list
mapping = []

# Iterate through each row
for _, row in nodes.iterrows():
    provinces = row['Coverage'].split(",")  # Split by comma
    for p in provinces:
        mapping.append({
            'Area': p,
            'Region': row['Region'],
            'Node ID': row['Node ID'],
            'Node': row['Node']
        })


# Create a new DataFrame
system = pd.DataFrame(mapping)

system = system.sort_values(by='Node ID').reset_index(drop=True)[['Area', 'Node', 'Node ID', 'Region']]

In [4]:
# Import the generator data

# Generalization of column names
col_names = {
    'No':'Index', 'Sistem':'System', 'Jenis':'Type', 'Pembangkit':'Name', 
    'Kapasitas(MW)':'Max Capacity', 'RE':'RE', 'ARED':'ARED', 'Pengembang':'Owner'}


datasets = {}

for i, path in enumerate(filepaths): 

    province_name = path[3:-4]  # extract province name from file name
    file_index = path[:2]

    raw_data = pd.read_csv(path, sep=';')
    raw_data = raw_data.iloc[:, ~raw_data.columns.str.contains('^Unnamed')].rename(columns=col_names)
    raw_data['Province'] =  path[3:-4] # add province column based on file name 
    raw_data['Index'] = file_index + "-" + raw_data['Index'].astype(str) # change index to include file index

    # Verify whether the total capacity matches the sum of the capacities in the dataset
    total_capacity_1 = int(capacity[capacity['Province'] == province_name]['TotalCapacity(MW)'].values[0])
    total_capacity_2 = int(raw_data['Max Capacity'].values.sum())

    if total_capacity_1 != total_capacity_2:
        #print(f"Warning: Total capacity mismatch for {province_name}. "
        #      f"Expected: {total_capacity_1}, Found: {total_capacity_2}")
        datasets[province_name] = raw_data
    else:
        #print(f"Total capacity matches for {province_name}: {total_capacity_1} MW")
        datasets[province_name] = raw_data


# Merge all the data
ds = pd.concat([data for _, data in datasets.items()])

### Add the Corresponding Node

In [5]:
for var in ['Node ID', 'Node']:
    ds[var] = ds['Area'].map(system.set_index('Area')[var] if system['Area'].values is not None else np.nan)

ds['Node ID'] = ds['Node ID'].astype('Int64')

# Data Cleaning

In [6]:
import unicodedata

# Functions for cleaning the data
def normalize_text(s):
    if pd.isnull(s):
        return None
    return unicodedata.normalize('NFKC', str(s)).strip().upper()


for var in ['Index', 'System', 'Name', 'Owner', 'Area']:
    ds[var] = ds[var].apply(normalize_text) 

ds['Name'] = ds['Name'].str.replace(r'(?<=\w)-(?=\d)', ' - ', regex=True)
ds['Name'] = ds['Name'].str.replace('#', '- ', regex=True)
ds['Name'] = ds['Name'].str.replace(r'\(FTP (\d)\)', r'(FTP\1)', regex=True)

ds['Status'] = ds['Status'].str.replace('COMITTED', 'COMMITTED')

for var in ['RE', 'ARED']:
    ds[var] = ds[var].replace(['-', ' '], 0, regex=True).astype(int)
    ds[var] = pd.to_datetime(ds[var].astype(str), format='%Y', errors='coerce')

In [7]:
roman_to_int = {
    'I':'1', 'II':'2', 'III':'3', 'IV':'4', 'V':'5', 'VI':'6', 'VII':'7', 'VIII':'8', 'IX':'9', 'X':'10'
}

for roman, intgr in roman_to_int.items():

    mask = ds['Name'].str.contains(fr'\b{roman}$', regex=True)

    df =  ds.loc[mask, 'Name'].str.extract(r'^(.*?)(?:\s+(\S+))?$')

    df[1] = df[1].str.replace(f'{roman}', f'{intgr}')

    ds.loc[mask, 'Name'] = df[0] + " " + df[1]

In [8]:
# Standardize the 'Owner' column
ds['Owner'] = ds['Owner'].replace({'PLN-SH':'PLN', 'SH-PLN':'PLN'})

In [9]:
# Create a standardization of ds types in 'Class' column
ds['Type'] = ds['Type'].replace({
    'PLTS+BESS': 'PLTS',
    'PLTBM': 'PLTBm',
    'PLTB+BESS': 'PLTB',
    'PS': 'Pump Storage',
    'PLTSA': 'PLTSa',
    'PLT Bio': 'PLTBio',
    #'PLTBg': 'PLTBiogas',
    'PLTU MT': 'PLTU',
    'PLTM': 'PLTA',
    'PLTGU/G': 'PLTGas',
    'PLTGU/G/MG/Relokasi/Sewa': 'PLTGas',
    'PLTG/GU': 'PLTGas',
    'PLTGU/G/MG': 'PLTGas'
})

var = 'Type'
# Show count for each type
gen_class = ds[var].groupby(ds[var]).count()

# Show sum of capacity for each type
gen_caps = ds.groupby(var)['Max Capacity'].sum()

pd.DataFrame({'Count': gen_class, 'Max Capacity': gen_caps}).sort_values(by='Count', ascending=False)

,Count,Max Capacity
Type,,
PLTS,344,17061.86181
PLTA,225,12230.21000
PLTP,111,5291.00000
PLTMG,101,2929.00000
PLTB,58,7187.50000
BESS,54,7568.20000
PLTU,45,6643.00000
PLTBm,30,423.30000
PLTGas,18,3645.00000


## Further Classification

In [10]:
ds['Class'] = np.where(
    (ds['Type'] == 'PLTU') & (ds['Owner'] == 'PLN'),
    'PLTU PLN',
    np.where(
        (ds['Type'] == 'PLTU') & (ds['Owner'] == 'IPP'),
        'PLTU IPP',
        ds['Type']
    )
)


# Show only the counts for the specified classes
# ds['Class'].value_counts().loc[['PLTU PLN', 'PLTU IPP', 'PLTU']]

# Divide the PLTA ds into PLTA Large and PLTA MediumMini
ds['Class'] = np.where(
    (ds['Type'] == 'PLTA') & (ds['Max Capacity'] >= 50.00),
    'PLTA Large',
    np.where(
        (ds['Type'] == 'PLTA') & (ds['Max Capacity'] < 50.00),
        'PLTA MediumMini',
        ds['Class']
    )
)

# Show only the counts for the specified classes
# ds['Class'].value_counts().loc[['PLTA Large', 'PLTA MediumMini']]

In [12]:
ds['Class'].unique()

array(['PLTA MediumMini', 'PLTBm', 'PLTB', 'BESS', 'PLTBg', 'PLTGas',
       'PLTA Large', 'PLTS', 'PLTP', 'PLTBio', 'PLTU', 'Pump Storage',
       'PLTMG', 'PLTN', 'PLTU IPP', 'PLTGU', 'PLTSa', 'PLTG', 'PLTU PLN',
       'PLTD', 'PLTAL'], dtype=object)

In [13]:
ds['Generator'] = ds['Node ID'].astype(str) + '_' + ds['Type'] + ' ' + ds['Name']

## Subset the RE Base Generator Data

In [14]:
ds_valid = ds[(ds['Node ID'].notna())] 

re_data = ds_valid[ds_valid['RE'].notna()].drop(columns=['ARED'])
re_data = re_data.rename(columns={'RE':'COD'})

In [174]:
# multiple_gen = ds_valid[ds_valid.groupby('Generator').cumcount() >= 1]['Generator']
# cond = ds_valid['Generator'].isin(multiple_gen)

# ds_valid[cond]['Generator']

# ds_valid.loc[cond, 'Generator'] = (
#     ds_valid.loc[cond, 'Generator'] +
#     ' - ' +
#     ds_valid.loc[cond].groupby('Generator').cumcount().add(1).astype(str)
# )


# Update the Database

### Load Existing Database

In [15]:
excel_path = r'C:\Users\binta\OneDrive\Documents\IESR\Generator\Generator_List_National_RUPTL 2025 update.xlsx'
xls = pd.ExcelFile(excel_path)
#print(sorted(xls.sheet_names))

database = {}

sheets = xls.sheet_names

for sheet in sheets:
    # print(f"{sheet}")
    open_db = pd.read_excel(excel_path, sheet_name=sheet)
    open_db = open_db.iloc[:, ~open_db.columns.str.contains('^Unnamed')]
    open_db['Class'] = sheet if sheet != 'PLTBiogas' else 'PLTBg'
    open_db['Status'] = 'EXISTING'
    open_db = open_db.rename(columns={'Region':'Node', 'Max capacity':'Max Capacity'})
    database[sheet] = open_db

In [16]:
db = pd.concat([data for _, data in database.items()])
db = db[~db['Generator'].isna()]
db = db.drop(columns=[' '])

db['Node ID'] = db['Node'].map(
    system.drop_duplicates(subset='Node').set_index('Node')['Node ID']
) if not system.empty else pd.Series(0, index=db.index)

# generalization 
db['Generator'] = db['Generator'].str.replace(r'\(FTP (\d)\)', r'(FTP\1)', regex=True)

### Update the Generator name

In [17]:
ds_valid = ds[(ds['Node ID'].notna())] 

re_data = ds_valid[ds_valid['RE'].notna()].drop(columns=['ARED'])
re_data = re_data.rename(columns={'RE':'COD'})

In [19]:
new_name = []

for idx, values in enumerate(re_data[['Generator', 'Class']].values):

    gen_name = values[0]
    gen_class = values[1]

    existing_count = db[
        (db['Class'] == gen_class) & 
        (db['Generator'].str.startswith(gen_name))
    ].shape[0]
    new_name.append(gen_name + ' - ' + str(existing_count+1))

re_data['Generator'] = new_name

In [20]:
re_data[['Base', 'Number']] = re_data['Generator'].str.extract(r'^(.*) - (\d+)$')
re_data['Number'] = re_data['Number'].astype(int)

# Step 2: Deteksi duplikat (berdasarkan Base + Number)
# dan buat penomoran lanjutan
used = {}  # menyimpan max number per base
new_names = []

for base, number in zip(re_data['Base'], re_data['Number']):
    key = (base, number)
    if key not in used:
        # Pertama kali muncul
        used[key] = 1
        max_num = used.get(base, number)
        used[base] = max(max_num, number)  # simpan max sejauh ini
        new_names.append(f"{base} - {number}")
    else:
        # Duplikat → beri nomor baru
        used[base] += 1
        new_names.append(f"{base} - {used[base]}")

# Step 3: Gantikan kolom Name
re_data['Generator'] = new_names

# Hapus kolom bantu
re_data = re_data.drop(columns=['Base', 'Number'])
re_data

,Index,System,Type,Name,Max Capacity,COD,Status,Owner,Area,Province,Node ID,Node,Class,Generator
0,01-1A,SUMATERA,PLTA,PEUSANGAN - 1,45.00,2025-01-01,Konstruksi,PLN,ACEH,Aceh,4,SUMBAGUT,PLTA MediumMini,4_PLTA PEUSANGAN - 1 - 1
1,01-1B,SUMATERA,PLTA,PEUSANGAN - 2,43.00,2025-01-01,Konstruksi,PLN,ACEH,Aceh,4,SUMBAGUT,PLTA MediumMini,4_PLTA PEUSANGAN - 2 - 1
2,01-2,SUMATERA,PLTBm,TANJUNG SEMANTO,9.80,2025-01-01,Konstruksi,IPP,ACEH,Aceh,4,SUMBAGUT,PLTBm,4_PLTBm TANJUNG SEMANTO - 2
3,01-3,SUMATERA,PLTB,BAYU SUMATERA,55.00,2026-01-01,Rencana,PLN,ACEH,Aceh,4,SUMBAGUT,PLTB,4_PLTB BAYU SUMATERA - 1
4,01-4,SUMATERA,PLTB,BAYU SUMATERA,55.00,2028-01-01,Rencana,PLN,ACEH,Aceh,4,SUMBAGUT,PLTB,4_PLTB BAYU SUMATERA - 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,34-75,TIMOR,PLTA,TIMOR (KUOTA) TERSEBAR,1.40,2029-01-01,Rencana,IPP,TIMOR,Nusa Tenggara Timur,15,TIMOR,PLTA MediumMini,15_PLTA TIMOR (KUOTA) TERSEBAR - 1
81,34-76,FLORES,PLTA,FLORES (KUOTA) TERSEBAR,0.22,2029-01-01,Rencana,IPP,FLORES,Nusa Tenggara Timur,16,FLORES,PLTA MediumMini,16_PLTA FLORES (KUOTA) TERSEBAR - 1
82,34-77,FLORES,PLTA,FLORES (KUOTA) TERSEBAR,16.00,2029-01-01,Rencana,IPP,FLORES,Nusa Tenggara Timur,16,FLORES,PLTA MediumMini,16_PLTA FLORES (KUOTA) TERSEBAR - 2
83,34-78,FLORES,PLTP,FLORES (KUOTA) TERSEBAR,40.00,2032-01-01,Rencana,IPP,FLORES,Nusa Tenggara Timur,16,FLORES,PLTP,16_PLTP FLORES (KUOTA) TERSEBAR - 1


In [21]:
generator_list = pd.concat([db, re_data])
col = 'Status'
cols = generator_list.columns.tolist()
cols.insert(1, cols.pop(cols.index(col)))

generator_list = generator_list[cols]
generator_list

,Generator,Status,Owner,Node,Province,Boiler Tech,Max Capacity,COD,Present year,Min Unit Built Year,...,Min down time,Run up,Max Volume,Min Volume,Node ID,Index,System,Type,Name,Area
0,16_PLTU ENDE (FTP1) - 1,EXISTING,PLN,FLORES,NTT,Stoker,7.00,2014-01-01 00:00:00,2023-01-01,2023-01-01,...,NaN,NaN,NaN,NaN,16,NaN,NaN,NaN,NaN,NaN
1,16_PLTU ENDE (FTP1) - 2,EXISTING,PLN,FLORES,NTT,Stoker,7.00,2014-01-01 00:00:00,2023-01-01,2023-01-01,...,NaN,NaN,NaN,NaN,16,NaN,NaN,NaN,NaN,NaN
2,23_PLTU SOFIFI - 1,EXISTING,PLN,HALMAHERA,Maluku Utara,Stoker,3.00,2021-01-01 00:00:00,2023-01-01,2023-01-01,...,NaN,NaN,NaN,NaN,23,NaN,NaN,NaN,NaN,NaN
3,23_PLTU SOFIFI - 2,EXISTING,PLN,HALMAHERA,Maluku Utara,Stoker,3.00,2021-01-01 00:00:00,2023-01-01,2023-01-01,...,NaN,NaN,NaN,NaN,23,NaN,NaN,NaN,NaN,NaN
4,1_PLTU INDRAMAYU - 1,EXISTING,PLN,JABAGBAR,Jawa Barat,PC,290.00,2010-01-01 00:00:00,2023-01-01,2023-01-01,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,15_PLTA TIMOR (KUOTA) TERSEBAR - 1,Rencana,IPP,TIMOR,Nusa Tenggara Timur,NaN,1.40,2029-01-01 00:00:00,NaT,NaT,...,NaN,NaN,NaN,NaN,15,34-75,TIMOR,PLTA,TIMOR (KUOTA) TERSEBAR,TIMOR
81,16_PLTA FLORES (KUOTA) TERSEBAR - 1,Rencana,IPP,FLORES,Nusa Tenggara Timur,NaN,0.22,2029-01-01 00:00:00,NaT,NaT,...,NaN,NaN,NaN,NaN,16,34-76,FLORES,PLTA,FLORES (KUOTA) TERSEBAR,FLORES
82,16_PLTA FLORES (KUOTA) TERSEBAR - 2,Rencana,IPP,FLORES,Nusa Tenggara Timur,NaN,16.00,2029-01-01 00:00:00,NaT,NaT,...,NaN,NaN,NaN,NaN,16,34-77,FLORES,PLTA,FLORES (KUOTA) TERSEBAR,FLORES
83,16_PLTP FLORES (KUOTA) TERSEBAR - 1,Rencana,IPP,FLORES,Nusa Tenggara Timur,NaN,40.00,2032-01-01 00:00:00,NaT,NaT,...,NaN,NaN,NaN,NaN,16,34-78,FLORES,PLTP,FLORES (KUOTA) TERSEBAR,FLORES


In [19]:

class_gen = generator_list['Class'].unique()

# Create Excel writer
with pd.ExcelWriter(r'C:\Users\binta\OneDrive\Documents\IESR\Generator\Results\Generator_List_Updated.xlsx', engine='xlsxwriter') as writer:
    for c in class_gen:
        subset = generator_list[generator_list['Class'] == c].sort_values(by=['Generator', 'Status'])
        sheet_name = str(c)[:31]  # Sheet names must be <=31 characters
        subset.to_excel(writer, sheet_name=sheet_name, index=False)

In [23]:
ds.to_excel(r'C:\Users\binta\OneDrive\Documents\IESR\Generator\Results\Data_RUPTL.xlsx')